In [2]:
!pip install nxpd

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install community

  Running setup.py bdist_wheel for community ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/0d/7e/f9/089b3c72d33c16b8d3edb17f64291511e5eea08b80e419faa7
Successfully built community
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install graph_tools

  Running setup.py bdist_wheel for pytess ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e6/a9/0c/deb8003b037eb99504cb256ea963c6880bafd36e4d0cce6ce0
Successfully built pytess
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections as mc
import networkx as nx
import nxpd as nxpd
import community as comm
import graph_tools as gt
from itertools import product
plt.style.use('ggplot')
%matplotlib inline

In [12]:
csv = 'df_filtered.csv'

In [13]:
#create G
def create_G(csv):
    sparsemat = create_sparsity_matrix(csv)
    features = sparsemat.columns
    g_mat = np.array(sparsemat)
    G = nx.Graph(g_mat)
    return features, G

In [ ]:
features, G = create_G(csv)
print(nx.info(G))

In [10]:
def plot_Gnodes(G):
    G.graph['rankdir'] = 'LR'
    return nxpd.draw(G, show='ipynb')

In [ ]:
plot_Gnodes(G)

In [ ]:
def plot_betweenness(G):
    pos = nx.spring_layout(G)
    betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)
    node_color = [20000.0 * G.degree(v) for v in G]
    node_size =  [v * 10000 for v in betCent.values()]
    plt.figure(figsize=(20,20))
    nx.draw_networkx(G, pos=pos, with_labels=False,
                 node_color=node_color,
                 node_size=node_size )
    plt.axis('off')

In [ ]:
def top5(G):
    betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)
    top5centrality = sorted(betCent, key=betCent.get, reverse=True)[:5]
    return features[top5centrality]

top5(G)

In [ ]:
import networkx as nx
import pandas as pd

def reduce_by_critical_mass(G, crit_mass=5000):
    """
    Reduces graph by removing nodes that have less than a specific number of
        followers (critical mass, default=5000).
    Args:
        G (nx.Graph()): Graph object, with all influencers and followers.
        crit_mass (Optional[int]): Critical mass following, default 5000.
    Returns:
        G_reduced (nx.Graph()): Graph object, containing only nodes with more
            than critical mass of followers.
    """
    G_reduced = G.copy()

    G_nodes = set(G.nodes())
    for node in G_nodes:
        if len(G.in_edges(node)) < crit_mass:
            G_reduced.remove_node(node)
    return G_reduced

def calc_community_influence_score(G,returned='list'):
    """
    Calculate and sort by community influence scores, which is eigenvector centrality
        for a graph (higher scores for higher eigenvector centralities).
    Args:
        G (nx.Graph()): Graph object.
        returned [Optional(list)]: 'list' or 'dict' will change data structure returned.
    Returns:
        sorted_influence_score (list of tuples): List of nodes sorted by eigenvector
            centrality.
    """

    c = nx.eigenvector_centrality(G)
    if returned == 'list':
        sorted_influence_score = (sorted(c.items(), key=lambda x: x[1], reverse=True))
        return sorted_influence_score
    else:
        return c

def calc_interaction_score(G, d, returned='list'):
    """
    Calculate and sort by interaction scores for reduced graph based
        on likes and followers (higher scores for larger likes to followers ratios).
    Args:
        G (nx.Graph()): Graph object.
        d (dict): Dictionary with user ids, number of likes, and number of followers.
        returned [Optional(list)]: 'list' or 'dict' will change data structure returned.
    Returns:
        sorted_interaction_score
    """
    interaction_score = {}

    for accession in G.nodes():
#         if accession == '':
#             interaction_score[''] = 0.05
#         else:
            interaction_score[accession] = (1.0*d[accession]['max_likes']/d[accession]['num_follow'])

    if returned == 'list':
        sorted_interaction_score = (sorted(interaction_score.iteritems(), key=lambda x: x[1], reverse=True))
        return sorted_interaction_score
    else:
        return interaction_score

def calc_authenticity_score(G, captions, returned='list'):
    """
    Calculate and sort by authenticity scores for reduced graph based
        on compound sentiment analysis score (higher scores for more negative posts).
    Args:
        G (nx.Graph()): Graph object.
        captions (dict): Dictionary with user ids and captions.
        returned [Optional(list)]: 'list' or 'dict' will change data structure returned.
    Returns:
        sorted_authenticity_score
    """
    analyzer = SentimentIntensityAnalyzer()
    top_influencers = G.nodes()
    top_influencers.remove("SelenaGomez")

    caption_sentiment = {}
    for user_id in top_influencers:
        for caption in captions[user_id]['caption']:
            vs = analyzer.polarity_scores(caption)
            if user_id in caption_sentiment:
                caption_sentiment[user_id].append(vs)
            else:
                caption_sentiment[user_id] = [vs]

    caption_sentiment_means = {}
    for user_id in caption_sentiment:
        df = pd.DataFrame(caption_sentiment[user_id])
        # Subtract compound score from 1 to penalize more positive captions
        caption_sentiment_means[user_id] = 1 - dict(df.mean())['compound']
    # Add Selena Gomez for test
    caption_sentiment_means["SelenaGomez"] = 0.5

    if returned == 'list':
        sorted_authenticity_score = (sorted(caption_sentiment_means.iteritems(), key=lambda x: x[1], reverse=True))
        return sorted_authenticity_score
    else:
        return caption_sentiment_means

def normalize_values(d):
    """
    Return dictionary with normalized values (0 to 1).
    Args:
        d (dict): Dictionary with user ids and raw scores.
    Returns:
        d_norm (dict): Dictionary with user ids and normalized scores.
    """
    minimum = min(d.iteritems(), key= lambda x: x[1])[1]
    maximum = max(d.iteritems(), key= lambda x: x[1])[1]

    d_norm = {}
    for user_id in d:
        d_norm[user_id] = (d[user_id] - minimum) / (maximum - minimum)

    return d_norm

def calc_overall_score(influence_score, interaction_score, authenticity_score):
    """
    Calculate overall scores and save all types of scores to one dictionary.
    Args:
        influence_score (dict): Dictionary with user ids and influence scores.
        interaction_score (dict): Dictionary with user ids and interaction scores.
        authenticity_score (dict): Dictionary with user ids and authenticity scores.
    Returns:
        scores (dict): Dictionary with all score types, and final scores.
    """
    scores = {}
    users = influence_score.keys()

    # Normalize scores dictionaries
    influence_normed = normalize_values(influence_score)
    interaction_normed = normalize_values(interaction_score)
    authenticity_normed = normalize_values(authenticity_score)

    # Add normalized scores types to scores
    for user_id in users:
        scores[user_id] = {'influence':None, 'interaction':None,
                            'authenticity':None, 'final':None}
        scores[user_id]['influence'] = influence_normed[user_id]
        scores[user_id]['interaction'] = interaction_normed[user_id]
        scores[user_id]['authenticity'] = authenticity_normed[user_id]

        # Add overall score to scores
        scores[user_id]['final'] = (0.6*influence_normed[user_id] +
                            0.3*interaction_normed[user_id] +
                            0.1*authenticity_normed[user_id])
    return scores

def find_top_influencers(G, filepath_interactions, filepath_captions):
    """
    Find top influencers based on a graph of influencers and followers,
    numbers of likes, and authenticity of posts.
    Args:
        G (nx.Graph()): Graph object, with all influencers and followers.
        filepath_interactions (str): Filepath to json with interactions dictionary.
        filepath_captions (str): Filepath to json with captions dictionary.
    Returns:
        scores (dict): Dictionary with all score types, and final scores.
    """
    # Reduce by in-degree
    G_reduced = reduce_by_critical_mass(G)
    interactions_dict = load_json(filepath_interactions)
    captions_dict = load_json(filepath_captions)

    # Calculate different scores
    influence_score = calc_community_influence_score(G_reduced, returned='dict')
    interaction_score = calc_interaction_score(G_reduced, interactions_dict, returned='dict')
    authenticity_score = calc_authenticity_score(G_reduced, captions_dict, returned='dict')

    # Calculate final scores
    scores = calc_overall_score(influence_score, interaction_score, authenticity_score)

    return scores